## Introduction
In this notebook we created the 3D-model for tumor segmentation! <br />

## Imports:
1. torch for model creation

In [1]:
import torch

## Model Definition
We can use our previously defined 2D-UNET architecture with some small changes:

1. Conv2d -> Conv3d
2. MaxPool2d -> MaxPool3d
3. "trilinear" upsampling method

In [2]:
class DoubleConv(torch.nn.Module):
    """
    Helper Class which implements the intermediate Convolutions
    """
    def __init__(self, in_channels, out_channels):
        
        super().__init__()
        self.step = torch.nn.Sequential(torch.nn.Conv3d(in_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv3d(out_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU())
        
    def forward(self, X):
        return self.step(X)

In [3]:
class UNet(torch.nn.Module):
    """
    3D U-Net:
      - 3 downs, 1 bottleneck, 3 ups
      - Skip connections by concatenation
      - Final 1-channel head for binary segmentation
    """
    def __init__(self):
        super().__init__()

        # ----- DOWN -----
        self.layer1 = DoubleConv(1,   32)
        self.layer2 = DoubleConv(32,  64)
        self.layer3 = DoubleConv(64,  128)
        self.layer4 = DoubleConv(128, 256)

        # ----- UP -----
        self.layer5 = DoubleConv(256 + 128, 128)
        self.layer6 = DoubleConv(128 +  64,  64)
        self.layer7 = DoubleConv( 64 +  32,  32)
        self.layer8 = torch.nn.Conv3d(32, 1, kernel_size=1)  # binary => 1 channel

        self.maxpool = torch.nn.MaxPool3d(kernel_size=2)

    def forward(self, x):
        # ----- Down -----
        x1  = self.layer1(x)
        x1m = self.maxpool(x1)

        x2  = self.layer2(x1m)
        x2m = self.maxpool(x2)

        x3  = self.layer3(x2m)
        x3m = self.maxpool(x3)

        # ----- Bottleneck -----
        x4 = self.layer4(x3m)

        # ----- Up 1 -----
        x5 = torch.nn.Upsample(scale_factor=2, mode="trilinear")(x4)
        x5 = torch.cat([x5, x3], dim=1)
        x5 = self.layer5(x5)

        # ----- Up 2 -----
        x6 = torch.nn.Upsample(scale_factor=2, mode="trilinear")(x5)
        x6 = torch.cat([x6, x2], dim=1)
        x6 = self.layer6(x6)

        # ----- Up 3 -----
        x7 = torch.nn.Upsample(scale_factor=2, mode="trilinear")(x6)
        x7 = torch.cat([x7, x1], dim=1)
        x7 = self.layer7(x7)

        # ----- Head -----
        return self.layer8(x7)

## Testing

In [4]:
model = UNet()

In [5]:
random_input = torch.randn(1, 1, 128, 128, 128)

In [6]:
with torch.no_grad():
    output = model(random_input)
assert output.shape == torch.Size([1, 1, 128, 128, 128])